In [2]:
pip install datasets

  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached typer_slim-0.21.1-py3-none-any.whl.metadata (16 kB)
   ---------------------------------------- 0.0/536.6 kB ? eta -:--:--
   ---------------------------------------- 536.6/536.6 kB 11.9 MB/s  0:00:00
Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl (2.9 MB)
   ---------------------------------------- 0.0/27.6 MB ? eta -:--:--
   --- ------------------------------------ 2.1/27.6 MB 13.8 MB/s eta 0:00:02
   ----- ---------------------------------- 3.9/27.6 MB 9.0 MB/s eta 0:00:03
   ------------- -------------------------- 9.4/27.6 MB 15.0 MB/s eta 0:00:02
   ---------------------- ----------------- 15.5/27.6 MB 18.7 MB/s eta 0:00:01
   ------------------------------ --------- 21.2/27.6 MB 20.6 MB/s eta 0:00:01
   ---------------------------------------  27.3/27.6 MB 21.8 MB/s eta 0:00:01
   ---------------------------------------- 27.6/27.6 MB 19.0 MB/s  0:00:01
Using cached typer_slim-0.21.1-py3-none

In [3]:
from datasets import load_dataset

ds = load_dataset("HeshamHaroon/Arabic_Function_Calling")

README.md: 0.00B [00:00, ?B/s]

c:\Users\Yaz00\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yaz00\.cache\huggingface\hub\datasets--HeshamHaroon--Arabic_Function_Calling. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.jsonl:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/45729 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5081 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
import json
from collections import defaultdict, Counter

ds = load_dataset("HeshamHaroon/Arabic_Function_Calling")
split = "train" if "train" in ds else list(ds.keys())[0]
d = ds[split]

TOOL_COL = "function_name"
ARGS_COL = "arguments"

def parse_args_strict(x):

    if isinstance(x, dict):
        return x
    if not isinstance(x, str):
        return None

    s = x.strip()
    if not (s.startswith("{") and s.endswith("}")):
        return None

    try:
        obj = json.loads(s)
    except json.JSONDecodeError:
        return None

    return obj if isinstance(obj, dict) else None

tool_total_calls = Counter()             
tool_key_counts = defaultdict(Counter)       

skipped = 0

for row in d:
    tool = row.get(TOOL_COL)
    raw = row.get(ARGS_COL)

    obj = parse_args_strict(raw)
    if tool is None or obj is None:
        skipped += 1
        continue

    tool_total_calls[tool] += 1
    for k in obj.keys():
        tool_key_counts[tool][k] += 1

# طباعة: لكل فنكشن، مجموع الاستدعاءات + تكرار كل key
for i, tool in enumerate(sorted(tool_total_calls.keys()), start=1):
    total = tool_total_calls[tool]
    print("\n" + "=" * 80)
    print(f"{i}) FUNCTION: {tool}  |  total_calls = {total}")
    print("=" * 80)

    # رتب المفاتيح بالأكثر تكرارًا
    for k, c in tool_key_counts[tool].most_common():
        print(f"  - {k}: {c}/{total}")

print(f"\nDone. Functions: {len(tool_total_calls)} | Rows skipped (no valid args): {skipped}")



1) FUNCTION:   |  total_calls = 4610

2) FUNCTION: book_doctor_appointment  |  total_calls = 2084
  - specialty: 2079/2084
  - city: 2076/2084
  - date: 1702/2084
  - day: 12/2084
  - location: 8/2084
  - hospital: 5/2084
  - time: 5/2084
  - doctor_name: 5/2084
  - time_frame: 2/2084
  - hospital_name: 1/2084
  - timeframe: 1/2084

3) FUNCTION: book_government_appointment  |  total_calls = 1145
  - city: 1141/1145
  - service_type: 1137/1145
  - id_number: 1136/1145
  - date: 4/1145
  - department: 2/1145
  - location: 2/1145
  - ministry: 1/1145
  - time: 1/1145
  - preferred_date: 1/1145
  - name: 1/1145

4) FUNCTION: calculate_customs  |  total_calls = 1290
  - value: 1288/1290
  - category: 1286/1290
  - country: 1286/1290
  - origin_country: 3/1290
  - item_type: 1/1290
  - currency: 1/1290
  - item_category: 1/1290
  - destination_city: 1/1290
  - order_value: 1/1290
  - destination: 1/1290

5) FUNCTION: calculate_end_of_service  |  total_calls = 984
  - salary: 984/984
  - yea

In [ ]:
import json
from collections import Counter

RENAME_RULES = {
  "book_doctor_appointment": {
    "location": ["city"],
    "hospital_name": ["hospital"],
    "date": ["time", "day", "time_frame", "timeframe"],
  },

  "book_government_appointment": {
    "location": ["city"],
    "date": ["time", "preferred_date"],
  },

  "calculate_customs": {
    "destination": ["destination_city"],
    "category": ["item_type", "item_category"],
    "value": ["order_value"],
    "country": ["origin_country"],
  },

  "calculate_loan": {
    "amount": ["principal", "loan_amount"],
    "duration": ["months", "duration_years", "loan_duration_years"],
  },

  "calculate_math": {
    "expression": ["equation"],
    "numbers": ["first_number", "second_number"],
  },

  "calculate_zakat": {
    "type": ["assets_type"],
  },

  "check_insurance_coverage": {
    "insurance_number": ["insurance_id"],
    "procedure": ["treatment"],
    "doctor_specialty": ["specialty"],
    "hospital": ["city"],
  },

  "check_iqama_status": {
    "iqama_number": ["id_number"],
    "person_name": ["name"],
  },

  "check_traffic_violations": {
    "id_number": ["iqama_number", "national_id"],
  },

  "get_gold_price": {
    "city": ["location"],
  },

  "get_hadith": {
    "query": ["topic"],
  },

  "get_weather": {
    "date": ["day", "days"],
  },

  "order_food": {
    "items": ["food_type"],
  },

  "search_flights": {
    "destination": ["destination_city", "arrival_city", "to_city"],
    "origin": ["departure_city", "from_city"],
    "date": ["departure_date", "departure_day", "day", "date_range"],
  },

  "search_hotels": {
    "city": ["location", "destination"],
    "check_in": ["check_in_date", "start_date"],
    "check_out": ["check_out_date"],
    "duration": ["nights", "dates", "date_range"],
  },

  "search_medications": {
    "city": ["location"],
    "condition": ["symptom"],
  },

  "search_umrah_packages": {
    "departure_city": ["city"],
  },

  "set_reminder": {
    "datetime": ["date", "day", "time"],
  },

  "track_shipment": {
    "tracking_number": ["shipment_number"],
    "destination_city": ["destination"],
    "city": ["origin", "source", "shipment_source", "location"],
  },

  "transfer_money": {
    "recipient_name": ["recipient"],
    "bank_name": ["bank"],
    "currency": ["currency_from"],
    "city": ["location", "destination", "destination_city"],
  },

  "translate_text": {
    "text": ["source_text"],
    "target_language": ["to_language", "to"],
  },
}


def groups_to_alias_map(groups: dict) -> dict:
    out = {}
    for tool, canon_map in groups.items():
        alias_map = {}
        for canonical, aliases in canon_map.items():
            for a in aliases:
                alias_map[a] = canonical
        out[tool] = alias_map
    return out

RENAME_RULES = groups_to_alias_map(RENAME_RULES)


CONFLICT_POLICY = "keep_first"

def resolve_conflict(existing, incoming, policy="keep_first"):
    if existing is None:
        return incoming
    if incoming is None:
        return existing
    if policy == "prefer_non_null":
        return existing if existing is not None else incoming
    
    return existing

def rename_keys_for_tool(tool, args_obj):
 
    rules = RENAME_RULES.get(tool)
    if not rules:
        return args_obj, False, 0

    new_args = {}
    changed = False
    conflicts = 0

    for k, v in args_obj.items():
        new_k = rules.get(k, k)
        if new_k != k:
            changed = True

        if new_k in new_args:
            conflicts += 1
            new_args[new_k] = resolve_conflict(new_args[new_k], v, policy=CONFLICT_POLICY)
        else:
            new_args[new_k] = v

    return new_args, changed, conflicts

updated_rows = []
changed_rows = 0
conflict_rows = 0
skipped = 0
tool_changed_counter = Counter()

for row in d:
    tool = row.get(TOOL_COL)
    raw = row.get(ARGS_COL)

    obj = parse_args_strict(raw)
    if tool is None or obj is None:
        skipped += 1
        updated_rows.append(row)
        continue

    new_obj, changed, conflicts = rename_keys_for_tool(tool, obj)

    if changed:
        changed_rows += 1
        tool_changed_counter[tool] += 1
        new_row = dict(row)
        new_row[ARGS_COL] = json.dumps(new_obj, ensure_ascii=False)
        updated_rows.append(new_row)
    else:
        updated_rows.append(row)

    if conflicts > 0:
        conflict_rows += 1

print(f"Done.")
print(f"Total rows: {len(d)} | skipped: {skipped}")
print(f"Changed rows: {changed_rows}")
print(f"Rows with key conflicts merged: {conflict_rows}")
print("\nTop changed functions:")
for t, c in tool_changed_counter.most_common(20):
    print(f"- {t}: {c}")



Done.
Total rows: 45729 | skipped: 0
Changed rows: 8506
Rows with key conflicts merged: 6

Top changed functions:
- get_weather: 2994
- book_doctor_appointment: 2079
- book_government_appointment: 1142
- calculate_loan: 1123
- search_umrah_packages: 1076
- search_hotels: 33
- search_flights: 14
- translate_text: 7
- track_shipment: 7
- transfer_money: 6
- calculate_customs: 4
- check_iqama_status: 3
- check_insurance_coverage: 3
- get_gold_price: 3
- search_medications: 3
- set_reminder: 2
- calculate_math: 2
- check_traffic_violations: 2
- get_hadith: 1
- order_food: 1


In [40]:
df_updated = pd.DataFrame(updated_rows)


In [42]:
from collections import defaultdict, Counter
import json

tool_total_calls2 = Counter()
tool_key_counts2 = defaultdict(Counter)
skipped2 = 0

for row in updated_rows:
    tool = row.get(TOOL_COL)
    raw = row.get(ARGS_COL)

    obj = parse_args_strict(raw)
    if tool is None or obj is None:
        skipped2 += 1
        continue

    tool_total_calls2[tool] += 1
    for k in obj.keys():
        tool_key_counts2[tool][k] += 1

for i, tool in enumerate(sorted(tool_total_calls2.keys()), start=1):
    total = tool_total_calls2[tool]
    print("\n" + "=" * 80)
    print(f"{i}) FUNCTION: {tool}  |  total_calls = {total}  (AFTER UPDATE)")
    print("=" * 80)
    for k, c in tool_key_counts2[tool].most_common():
        print(f"  - {k}: {c}/{total}")

print(f"\nDone (AFTER). Functions: {len(tool_total_calls2)} | Rows skipped: {skipped2}")



1) FUNCTION:   |  total_calls = 4610  (AFTER UPDATE)

2) FUNCTION: book_doctor_appointment  |  total_calls = 2084  (AFTER UPDATE)
  - location: 2084/2084
  - specialty: 2079/2084
  - date: 1719/2084
  - hospital_name: 6/2084
  - doctor_name: 5/2084

3) FUNCTION: book_government_appointment  |  total_calls = 1145  (AFTER UPDATE)
  - location: 1143/1145
  - service_type: 1137/1145
  - id_number: 1136/1145
  - date: 5/1145
  - department: 2/1145
  - ministry: 1/1145
  - name: 1/1145

4) FUNCTION: calculate_customs  |  total_calls = 1290  (AFTER UPDATE)
  - value: 1289/1290
  - country: 1289/1290
  - category: 1288/1290
  - destination: 2/1290
  - currency: 1/1290

5) FUNCTION: calculate_end_of_service  |  total_calls = 984  (AFTER UPDATE)
  - salary: 984/984
  - years: 984/984
  - termination_type: 984/984

6) FUNCTION: calculate_loan  |  total_calls = 1124  (AFTER UPDATE)
  - amount: 1124/1124
  - duration: 1123/1124
  - rate: 1121/1124
  - currency: 1/1124
  - country: 1/1124

7) FUNCT

In [43]:
df_updated.head()

,id,query_ar,query_en,function_name,arguments,dialect,domain,requires_function
0,gen_18986,أريد إنشاء تذكير بعنوان اجتماع عمل في الساعة ا...,I want to set a reminder titled Business Meeti...,set_reminder,"{""title"": ""اجتماع عمل"", ""datetime"": ""2023-10-1...",MSA,utilities,True
1,gen_32894,ممكن تترجملي هاي الجملة للإنجليزي؟,Can you translate this sentence to English?,translate_text,"{""text"": ""هاي الجملة"", ""target_language"": ""en""}",Levantine,utilities,True
2,gen_18262,بدي احول ٣٠٠٠ ليرة لبنانية لعمار إبراهيم رقم ا...,I want to transfer 3000 Lebanese pounds to Amm...,transfer_money,"{""amount"": 3000, ""currency"": ""LBP"", ""recipient...",Levantine,banking_finance,True
3,gen_29590,الجو مشمس في شرم الشيخ النهارده؟,Is it sunny in Sharm El-Sheikh today?,get_weather,"{""city"": ""شرم الشيخ"", ""date"": 0}",Egyptian,weather,True
4,gen_08985,ترجملّي هالنص للإسباني: كيفك اليوم؟,Translate this text to Spanish: How are you to...,translate_text,"{""text"": ""كيفك اليوم؟"", ""target_language"": ""Sp...",Levantine,utilities,True


In [58]:
(df_updated["requires_function"] == False).sum()


np.int64(4554)